In [ ]:
from helper import *

path='../reports/corpus.xlsx'

description = get_column_values(path, 'descriptions_measures', 'A')


targets = get_column_values(path, 'descriptions_measures', 'B')

similarities_df = similarities_df(docs=description,group=targets,encoding_method='tfidf')

similarities_df.head()

In [ ]:
from helper import *

Q1 = 'Create a report that shows the number of settlement applications approved during the month, their approved amount,\
      the written off balance, the average days to approval, their average and median duration and their entry principal and balance, \
        for September 2020 per DCA and application type. The report should be produced on Earth portfolio. '

path = '../reports/corpus.xlsx'
targets = get_column_values(path, 'descriptions_measures', 'B')
encoding_method = 'word2vec'
version = 'word2vec-google-news-300'




similarities_dict = get_similarity_scores(text=Q1,documents=targets,encoding_method=encoding_method,version=version)

top_10 = get_first_n_keys(similarities_dict, 10)


top_10

In [1]:
from preprocessing import *
from helper import *

Q1 = 'Create a report that shows the number of settlement applications approved during the month, their approved amount,\
      the written off balance, the average days to approval, their average and median duration and their entry principal and balance, \
        for September 2020 per DCA and application type. The report should be produced on Earth portfolio. '
processed_text = preprocess_text(Q1)



w2v = api.load("word2vec-google-news-300")
vocab = w2v.index_to_key
words = Q1.split()


count = sum(1 for word in processed_text if word in w2v)
existing_words = [word for word in processed_text if word in w2v]
difference = len(words) - count

pr, orig = get_unique_items(words, processed_text)

pr

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\avitsas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\avitsas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\avitsas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
c:\Users\avitsas\Anaconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['Create',
 'a',
 'that',
 'shows',
 'the',
 'of',
 'applications',
 'during',
 'the',
 'month,',
 'their',
 'amount,',
 'the',
 'off',
 'balance,',
 'the',
 'days',
 'to',
 'approval,',
 'their',
 'and',
 'and',
 'their',
 'and',
 'balance,',
 'for',
 'September',
 '2020',
 'DCA',
 'and',
 'type.',
 'The',
 'should',
 'be',
 'on',
 'Earth',
 'portfolio.']

In [1]:
from preprocessing import *

Q1 = 'Create a report that shows the number of settlement applications approved during the month, their approved amount,\
      the written off balance, the average days to approval, their average and median duration and their entry principal and balance, \
        for September 2020 per DCA and application type. The report should be produced on Earth portfolio. '

processed_text = preprocess_text(Q1)

processed_text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\avitsas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\avitsas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\avitsas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['create',
 'report',
 'show',
 'number',
 'settlement',
 'application',
 'approved',
 'month',
 'approved',
 'amount',
 'written',
 'balance',
 'average',
 'day',
 'approval',
 'average',
 'median',
 'duration',
 'entry',
 'principal',
 'balance',
 'september',
 'per',
 'dca',
 'application',
 'type',
 'report',
 'produced',
 'earth',
 'portfolio']